<span style="font-size:24px; font-family:'Roboto'; font-weight:bold;">
Script to make SHD files from CHD files.
</span><br>
iMOD doesn't accept nan values as SHD, but CHD files have nan (for cells where the layer thickness is 0, thus the model is inactive).<br>
We'll fill them with interpolated values. The values don't really matter.<br>
Some layers are inactive (not due to IBOUND, but cause Thk=0) in large parts of the model area. We'll fill (interpolate between, then extrapolate) the NaN values to the extend of L1 (which is active everywhere). Some layersare completely inactive. Those will be filled with values copied from the layer above.

This script can be improved when I have time. Then I should convert it to a .py file.

## 0.0. Libraries

In [1]:
import WS_Mdl as WS # import functions defined for WS_Mdl (by me)
import WS_Mdl.utils as U  # import utility functions defined for WS_Mdl (by me)

In [2]:
import os
from os import listdir as LD, makedirs as MDs
from os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE 
import imod
import xarray as xr
import numpy as np

In [3]:
import matplotlib.pyplot as plt

## 0.1. Options

In [4]:
MdlN_S = 'NBr34'
MdlN_B = 'NBr5'

In [5]:
date_B = '20171228'
date_S = date_B

In [6]:
Mdl = ''.join([c for c in MdlN_S if not c.isdigit()])

In [7]:
d_Pa = U.get_MdlN_paths(MdlN_S)

🟢 - NBr34 paths extracted from RunLog and returned as dictionary with keys:
Mdl, MdlN, Pa_Mdl, Smk_temp, PoP, INI, BAT, PRJ, Smk, Sim, Pa_MdlN, LST_Sim, LST_Mdl, NAM_Sim, NAM_Mdl, Out_HD, PoP_Out_MdlN, MM, Mdl_B, MdlN_B, Pa_Mdl_B, Smk_temp_B, PoP_B, INI_B, BAT_B, PRJ_B, Smk_B, Sim_B, Pa_MdlN_B, LST_Sim_B, LST_Mdl_B, NAM_Sim_B, NAM_Mdl_B, Out_HD_B, PoP_Out_MdlN_B, MM_B


c:\Users\Karam014\OneDrive - Universiteit Utrecht\WS_Mdl\code\.pixi\envs\default\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [8]:
Pa_CHD = rf"C:\OD\WS_Mdl\models\NBr\In\CHD\{MdlN_B}"
Pa_SHD = rf"C:\OD\WS_Mdl\models\NBr\In\SHD\{MdlN_S}"

# 1. Read CHD, fill (interpolate), save as SHD 

In [9]:
l_CHD = [PJ(Pa_CHD, i) for i in LD(Pa_CHD) if (f'HEAD_{date_B}_' in i) and ('.idf' in i.lower())]

In [10]:
DA_CHD = imod.formats.idf.open(l_CHD, pattern='{name}_L{layer}_NBr5')

In [11]:
DA_CHD

<xarray.DataArray 'head_20171228' (layer: 37, y: 520, x: 740)> Size: 114MB
dask.array<stack, shape=(37, 520, 740), dtype=float64, chunksize=(1, 520, 740), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) float64 6kB 6.705e+04 6.715e+04 ... 1.408e+05 1.41e+05
  * y        (y) float64 4kB 4.21e+05 4.208e+05 4.208e+05 ... 3.692e+05 3.69e+05
    dx       float64 8B 100.0
    dy       float64 8B -100.0
  * layer    (layer) int64 296B 1 2 3 4 5 6 7 8 9 ... 29 30 31 32 33 34 35 36 37

In [12]:
# 1. Sort coordinates to allow interpolation
reversed_y = not np.all(np.diff(DA_CHD.y.values) > 0)
reversed_x = not np.all(np.diff(DA_CHD.x.values) > 0)

In [13]:
if reversed_y:
    DA_CHD = DA_CHD.sortby("y")
if reversed_x:
    DA_CHD = DA_CHD.sortby("x")

In [14]:
# fig, ax = plt.subplots()
# img = ax.imshow(mask_valid, cmap="gray")
# cbar = fig.colorbar(img, ax=ax)

In [15]:
mask_valid = ~DA_CHD.isel(layer=0).isnull() # Get valid mask from layer 0

In [16]:
# 2. Fill missing values with inter/extrapolation.
DA_CHD_interp_list = []
for i in range(DA_CHD.sizes["layer"]):
    layer_i = DA_CHD.isel(layer=i)
    
    if layer_i.isnull().all():
        filled = DA_CHD_interp_list[-1]  # if the layer is all NaN use previous L
    else:
        filled = (
            layer_i.interpolate_na(dim="y", method="linear")
                    .interpolate_na(dim="x", method="linear")
                    .fillna(layer_i.ffill("y").bfill("y").ffill("x").bfill("x"))
                    .where(mask_valid)
        )
    DA_CHD_interp_list.append(filled.assign_coords(layer=DA_CHD.layer.isel(layer=i)))

DA_CHD_interp = xr.concat(DA_CHD_interp_list, dim="layer")
DA_CHD_interp["layer"] = DA_CHD["layer"]

In [17]:
# A = DA_CHD_interp.isel(layer=1-1).squeeze().sortby('y', ascending=False)
# fig, ax = plt.subplots()
# img = ax.imshow(A, cmap="inferno")
# cbar = fig.colorbar(img, ax=ax)

In [18]:
# 3. Reverse coords back to original orientation
if reversed_y:
    DA_CHD_interp = DA_CHD_interp.sortby("y", ascending=False)
if reversed_x:
    DA_CHD_interp = DA_CHD_interp.sortby("x", ascending=False)

In [19]:
DA_CHD_interp = DA_CHD_interp.expand_dims(name=[f"SHD_{date_S}"])

In [20]:
# imod.idf.save(Pa_SHD + "/dummy.idf", DA_CHD_interp, pattern="{name}_L{layer}_NBr8.IDF")

# 2. Write SHD block

In [23]:
for i in range(37):
    print(fr" 1,2, {i+1:03},   1.000000    ,   0.000000    ,  -999.9900    , '..\..\In\SHD\{MdlN_S}\SHD_{date_B}_L{i+1}_{MdlN_S}.IDF' >>> (shd) starting heads (idf) <<<")

 1,2, 001,   1.000000    ,   0.000000    ,  -999.9900    , '..\..\In\SHD\NBr34\SHD_20171228_L1_NBr34.IDF' >>> (shd) starting heads (idf) <<<
 1,2, 002,   1.000000    ,   0.000000    ,  -999.9900    , '..\..\In\SHD\NBr34\SHD_20171228_L2_NBr34.IDF' >>> (shd) starting heads (idf) <<<
 1,2, 003,   1.000000    ,   0.000000    ,  -999.9900    , '..\..\In\SHD\NBr34\SHD_20171228_L3_NBr34.IDF' >>> (shd) starting heads (idf) <<<
 1,2, 004,   1.000000    ,   0.000000    ,  -999.9900    , '..\..\In\SHD\NBr34\SHD_20171228_L4_NBr34.IDF' >>> (shd) starting heads (idf) <<<
 1,2, 005,   1.000000    ,   0.000000    ,  -999.9900    , '..\..\In\SHD\NBr34\SHD_20171228_L5_NBr34.IDF' >>> (shd) starting heads (idf) <<<
 1,2, 006,   1.000000    ,   0.000000    ,  -999.9900    , '..\..\In\SHD\NBr34\SHD_20171228_L6_NBr34.IDF' >>> (shd) starting heads (idf) <<<
 1,2, 007,   1.000000    ,   0.000000    ,  -999.9900    , '..\..\In\SHD\NBr34\SHD_20171228_L7_NBr34.IDF' >>> (shd) starting heads (idf) <<<
 1,2, 008,   

# 3. Write metadata file in the same folder

In [24]:
os.getcwd()

'c:\\OD\\WS_Mdl\\code\\Jupyter\\PrP\\CHD_to_SHD'

In [26]:
with open(PJ(Pa_SHD, '_metadata.txt'), 'w') as f:
    f.write(fr"This file was produced with 'c:\\OD\\WS_Mdl\\code\\PrP\\CHD_to_SHD\CHD_to_SHD.ipynb' cause iMOD won't accept SHD data with nan values. (Otherwise I would have used the CHD data directly.)")